# Choice of location for the well

Let's say you work for the mining company GlavNeft. We need to decide where to drill a new well.

You have been provided with oil samples in three regions: in each of 10,000 fields, where the quality of oil and the volume of its reserves have been measured. Build a machine learning model to help determine the region where mining will bring the most profit. Analyze possible profits and risks using the *Bootstrap.* technique

Steps to choose a location:

- In the selected region, they are looking for deposits, for each, the values ​​of the signs are determined;
- Build a model and estimate the volume of reserves;
- Select the deposits with the highest value estimates. The number of fields depends on the company's budget and the cost of developing one well;
- The profit is equal to the total profit of the selected fields.

## Loading and preparing data

In [1]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

import warnings
warnings.filterwarnings('ignore')

In [2]:
#loading files
path ='/Users/vzuga/Documents/jupyter/'
try:
    df1 = pd.read_csv('geo_data_0.csv') #Region 1
    df2 = pd.read_csv('geo_data_1.csv') #Region 2
    df3 = pd.read_csv('geo_data_2.csv') #Region 3
except:
    df1 = pd.read_csv(path+'/datasets/geo_data_0.csv')
    df2 = pd.read_csv(path+'/datasets/geo_data_1.csv')
    df3 = pd.read_csv(path+'/datasets/geo_data_2.csv')

I will write a function that will show brief information about the file, display the first 5 lines, check for duplicates and repetitions in the well ID.

In [3]:
def data_check(data):
    data.info()
    print()
    display(data.head())
    print()
    print('Duplicates:', data.duplicated().sum())
    print()
    print('Number of unique ID values:',len(data['id'].unique()))  

### Region 1

In [4]:
data_check(df1)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB



,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647



Duplicates: 0

Number of unique ID values: 99990


### Region 2

In [5]:
data_check(df2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB



,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305



Duplicates: 0

Number of unique ID values: 99996


### Region 3

In [6]:
data_check(df3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB



,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746



Duplicates: 0

Number of unique ID values: 99996


### Conclusions:

* No gaps or obvious duplicates were found in the data
* there is a small number of repetitions in the ID, but there are very few of them, I will ignore them
* I will not make any changes to the data

## Train and validate the model

### Splitting data into samples

I will divide the data into 2 parts in a ratio of 3: 1 into training and validation samples.

In [7]:
#extract features and target
features1 = df1.drop(['id', 'product'], axis=1)
target1 = df1['product']

features2 = df2.drop(['id', 'product'], axis=1)
target2 = df2['product']

features3 = df3.drop(['id', 'product'], axis=1)
target3 = df3['product']

In [8]:
#creating samples
features_train1, features_valid1, target_train1, target_valid1 = train_test_split(
    features1, target1, test_size=.25, random_state=12345)

features_train2, features_valid2, target_train2, target_valid2 = train_test_split(
    features2, target2, test_size=.25, random_state=12345)

features_train3, features_valid3, target_train3, target_valid3 = train_test_split(
    features3, target3, test_size=.25, random_state=12345)

In [9]:
#size check
print('features')
print(len(features_train1), len(features_valid1))
print(len(features_train2), len(features_valid2))
print(len(features_train3), len(features_valid3))
print()
print('targets')
print(len(target_train1), len(target_valid1))
print(len(target_train2), len(target_valid2))
print(len(target_train3), len(target_valid3))

features
75000 25000
75000 25000
75000 25000

targets
75000 25000
75000 25000
75000 25000


### Feature scaling

In [10]:
#numeric features
numeric = ['f0', 'f1', 'f2']

In [11]:
# Region 1
scaler1 = StandardScaler()
scaler1.fit(features_train1[numeric])
features_train1[numeric] = scaler1.transform(features_train1[numeric])
features_valid1[numeric] = scaler1.transform(features_valid1[numeric])

# Region 2
scaler2 = StandardScaler()
scaler2.fit(features_train2[numeric])
features_train2[numeric] = scaler2.transform(features_train2[numeric])
features_valid2[numeric] = scaler2.transform(features_valid2[numeric])

# Region 3
scaler3 = StandardScaler()
scaler3.fit(features_train3[numeric])
features_train3[numeric] = scaler3.transform(features_train3[numeric])
features_valid3[numeric] = scaler3.transform(features_valid3[numeric])

In [12]:
features_train3.head()

,f0,f1,f2
27212,-0.526160,0.776329,-0.400793
7866,-0.889625,-0.404070,-1.222936
62041,-1.133984,0.208576,0.296765
70185,1.227045,1.570166,-0.764556
82230,-0.194289,0.878312,0.840821


### Model Training

In [13]:
model1 = LinearRegression()
model1.fit(features_train1, target_train1)
predicted_valid1 = model1.predict(features_valid1)

model2 = LinearRegression()
model2.fit(features_train2, target_train2)
predicted_valid2 = model2.predict(features_valid2)

model3 = LinearRegression()
model3.fit(features_train3, target_train3)
predicted_valid3 = model3.predict(features_valid3)

In [14]:
rmse1 = mean_squared_error(target_valid1, predicted_valid1)**.5

rmse2 = mean_squared_error(target_valid2, predicted_valid2)**.5

rmse3 = mean_squared_error(target_valid3, predicted_valid3)**.5

### Conclusions

In [15]:
mean_rmse = pd.DataFrame(data={'region':['Region 1', 'Region 2', 'Region 3'],                               
'target_mean':[target_valid1.mean(), target_valid2.mean(), target_valid3.mean()],
'predicted_mean':[predicted_valid1.mean(), predicted_valid1.mean(), predicted_valid1.mean()],
'RMSE':[rmse1, rmse2, rmse3]})

mean_rmse

,region,target_mean,predicted_mean,RMSE
0,Region 1,92.078597,92.592568,37.579422
1,Region 2,68.723136,92.592568,0.893099
2,Region 3,94.884233,92.592568,40.029709


* The models did a good job of predicting the average stock of raw materials.
* The quality of models for the first and third regions are low.
* Since further model predictions are only used to select wells with the highest estimated values, the absolute values of the predicted reserves are not so important.

## Preparation for profit calculation

* The budget for the development of wells in the region is 10 billion rubles
* Income from each unit of the product is 450 thousand rubles
* When exploring a region, 200 best wells are selected for development.

In [16]:
BUDGET = 1e10
INCOME = 4.5e5
N_WELL = 200 #final number of wells
SCOUT_WELL = 500 #number of exploration wells

Sufficient volume of raw materials for break-even development of a new well:

In [17]:
product_required = BUDGET/INCOME/N_WELL
print('The volume of raw materials for break-even development:', int(product_required), 'thousand barrels')
print()
print('Average stock in each region:')
display(mean_rmse[['region', 'target_mean']])

The volume of raw materials for break-even development: 111 thousand barrels

Average stock in each region:


,region,target_mean
0,Region 1,92.078597
1,Region 2,68.723136
2,Region 3,94.884233


In [18]:
#average production from the top 200 wells from initial data
df1.sort_values(by='product', ascending=False).head(N_WELL)['product'].mean()

184.83373964536008

### Conclusions:

* Sufficient volume of raw materials for break-even development of one well with the current budget and the number of wells is 111 thousand barrels.
* The average stock in each region is less than sufficient.
* If we select the best 200 wells in each region, then there will be enough of them for break-even production.

## Profit and Risk Calculation

### Profit calculation function

I will write a function to calculate profit based on the selected number of wells and model predictions.

In [19]:
def profit(prediction, target):
    return target[prediction.sort_values(ascending=False).index].head(N_WELL).sum()*INCOME - BUDGET
#the function receives pd.Series as input, so that in the future it does not wrap np.array twice in a Series

In [20]:
profit(pd.Series(predicted_valid1), pd.Series(target_valid1).reset_index(drop=True))

3320826043.1398506

### Risks and rewards for each region
I'll use the Bootstrap technique with 1000 samples to find the profit distribution.

In [21]:
def profit_bootstrap(prediction, target):
    state = np.random.RandomState(12345)
    profit_values = []
    prediction = pd.Series(prediction)
    target = pd.Series(target).reset_index(drop=True)
    for i in range(1000):
        prediction_subsample = prediction.sample(n=SCOUT_WELL, replace=True, random_state=state)
        profit_values.append(profit(prediction_subsample, target))
    return profit_values

### Region 1

In [22]:
region1 = pd.Series(profit_bootstrap(predicted_valid1, target_valid1))

In [23]:
print(f'Average profit: {region1.mean():e}')
print(f'Confidence interval: ({region1.quantile(.025):e}, {region1.quantile(.975):e})')
print(f'Risk of loss {region1.lt(0).mean():.1%}')

Average profit: 3.961650e+08
Confidence interval: (-1.112155e+08, 9.097669e+08)
Risk of loss 6.9%


### Region 2

In [24]:
region2 = pd.Series(profit_bootstrap(predicted_valid2, target_valid2))

In [25]:
print(f'Average profit: {region2.mean():e}')
print(f'Confidence interval: ({region2.quantile(.025):e}, {region2.quantile(.975):e})')
print(f'Risk of loss {region2.lt(0).mean():.1%}')

Average profit: 4.560451e+08
Confidence interval: (3.382051e+07, 8.522895e+08)
Risk of loss 1.5%


### Region 3

In [26]:
region3 = pd.Series(profit_bootstrap(predicted_valid3, target_valid3))

In [27]:
print(f'Average profit: {region3.mean():e}')
print(f'Confidence interval: ({region3.quantile(.025):e}, {region3.quantile(.975):e})')
print(f'Risk of loss {region3.lt(0).mean():.1%}')

Average profit: 4.044039e+08
Confidence interval: (-1.633504e+08, 9.503596e+08)
Risk of loss 7.6%


### Conclusion:

* models were built to determine the region with the highest profit
* using the bootstrap technique, the possible profit, confidence interval and risks were calculated
* the highest returns and lowest risks were found in the second region
* according to the results of the study, region No. 2 is proposed for development